In [1]:
import os
import PySpin
from PIL import Image
import datetime
import matplotlib.pyplot as plt
import sys
import keyboard
import time
from datetime import datetime
import math
import numpy as np
import scipy.ndimage as ndi
import cv2
import warnings
from skimage.transform import hough_line, hough_line_peaks
from scipy.ndimage import sobel
from scipy.ndimage import convolve

In [2]:
# not used
# gets rid of dark pixels on the edges if extra stripes are include in the image
def crop_dark_edges(image):
    dark = True
    top = 0
    bottom = -1
    left = 0
    right = -1
    while (dark and top < image.shape[0]):
        line = image[top]
        if (np.average(line) < 80):
            top +=1
        else:
            dark = False
    dark = True
    while (dark and abs(bottom) < image.shape[0]//2):
        line = image[bottom]
        if (np.average(line) < 80):
            bottom -=1
        else:
            dark = False
    dark = True
    while (dark and left < image.shape[1]):
        line = image[:,left]
        if (np.average(line) < 80):
            left += 1
        else:
            dark = False
    dark = True
    while (dark and abs(right)< image.shape[1]//2):
        line = image[:,right]
        if (np.average(line)< 80):
            right -=2
        else:
            dark = False
    return image[top:bottom,left:right]

In [3]:
def horizontal_sobel(image):
    image = image.astype(np.float32)
    sobelled = sobel(image, 0) # apply a horizontal sobel filter: looking for vertical stripes
    #normalize the image
    sobelled = np.abs(sobelled)
    biggest = np.max(sobelled) 
    sobelled = (sobelled/biggest)*255
    return sobelled

In [4]:
# returns a binary image given an image that has been edge filtered and the threshold at which something counts as white
def get_edges(edgeimage, threshold):
    edgeimage = edgeimage > threshold
    edgeimage = edgeimage * 255
    return edgeimage

In [5]:
# uses probabilistic hough transform
# using canny and propabilistic hough is faster than sobel and normal Hough, but tends to crop too widely
def find_hough_lines_p(image):
    endpoints = cv2.HoughLinesP(image, 1, np.pi/60, 40, minLineLength=15, maxLineGap=100)
    return endpoints

In [6]:
# use hough transform to get the lines on an image
def find_hough_lines(binaryimage):
    # only look for horizontal lines
    testangles = np.linspace(1.47,1.67, 20, endpoint = True)
    h, theta, d = hough_line(binaryimage, theta=testangles)
    # get the 20 lines with highest intensity
    accumulator, angles, rho = hough_line_peaks(h,theta,d, num_peaks = 25)
    # if no lines are found using num_peaks parameter, use the threshold parameter to try and find lines
    if angles.shape[0] == 0:
        return find_hough_lines_threshold(h, theta, d)
    else:
        return angles, rho

In [7]:
# if no lines are found, lower the threshold manually to get lines
def find_hough_lines_threshold(h, theta, d):
    base_threshold = 0
    accumutor, angles, rho = hough_line_peaks(h, theta, d, threshold = base_threshold)
    num_lines = angles.shape[0]
    # keep decreasing thresholds until less than fifteen lines are found
    while num_lines > 15:
        base_threshold += num_lines/8
        accumulator, angles, rho = hough_line_peaks(h, theta, d, threshold = base_threshold)
        # if the number of lines found suddenly decreases to 0, go back to the old threshold and just return that result regardless of how many lines there were
        if angles.shape[0] == 0:
            base_threshold -= num_lines/8
            accumulator, angles, rho = hough_line_peaks(h, theta, d, threshold = base_threshold)
            return angles, rho
        print(base_threshold)
        num_lines = angles.shape[0]
        #print(num_lines)
    #print("threshold:" + str(base_threshold))
    return angles, rho

In [8]:
def get_crop_from_lines(angles, rho, image):
    if len(rho) < 2:
        print("Not enough lines in the right direction were found")
    # takes the 2 lines that are closest to the edges, and returns the horizontal indices
    return (np.min(rho), np.max(rho))

In [9]:
# takes the endpoints given from the probabilistic Hough lines, finds the maximum and minimum y values
def get_crop_from_lines_p(endpoints):
    horizontal_lines = []
    print(horizontal_lines)
    for i in range(endpoints.shape[0]):
        current_line = endpoints[i,0]
        x1 = current_line[0]
        x2 = current_line[2]
        y1 = current_line[1]
        y2 = current_line[3]
        angle = math.atan2(y2 - y1, x2 - x1)
        #print(angle)
        # Check if the line is approximately horizontal (angle within a range)
        if  -0.25 <= angle <= 0.25:
            horizontal_lines.append(current_line)
    horizontal_lines = np.array(horizontal_lines)
    y_values = horizontal_lines[:,1]
    np.append(y_values, horizontal_lines[:,3])
    min_y = np.min(y_values)
    max_y = np.max(y_values)
    return min_y, max_y

In [10]:
# takes in a greyscale image and finds the striped region of interest for dof calculations
# assumes that it is looking for vertical lines (rotates image)
def get_cropped_region(image):
    #image = crop_dark_edges(image)
    if image.size == 0:
        print("Image too dark. Try increasing exposure time.")
        return
    filtered = horizontal_sobel(image)
    edgethreshold = np.average(filtered) * 6
    edges = get_edges(filtered, edgethreshold)
    angles, rho = find_hough_lines(edges)
    if angles.shape[0] == 0:
        print("No lines found")
        return
    boundaries = get_crop_from_lines(angles, rho, edges)
    #print(boundaries)
    if (boundaries is None):
        print("Could not find proper region from lines")
        return
    # give a little space in the region
    lower = int(0.95 * boundaries[0])
    higher = int(1.05 * boundaries[1])
    croppedimage = image[lower:higher,:]
    return croppedimage

In [11]:
def get_cropped_region_p(image):
    #image = crop_dark_edges(image)
    if image.size == 0:
        print("Image too dark. Try increasing exposure time.")
        return
    filtered = cv2.Canny(image, 20, 80)
    xy = find_hough_lines_p(filtered)
    #print (angles)
    #print(rho)
    if xy is None:
        print('No lines found')
        return
    boundaries = get_crop_from_lines_p(xy)
    print(boundaries)
    if (boundaries is None):
        print("Could not find proper region from lines")
        return
    # give a little space in the region
    lower = int(0.95 * boundaries[0])
    higher = int(1.05 * boundaries[1])
    croppedimage = image[lower:higher,:]
    return croppedimage

In [12]:
def bin_image(im, bin_size):
    shape = im.shape
    print(shape)
    while (len(im) % bin_size) != 0:
        im = im[:((len(im) // bin_size)*bin_size)]
    print(im.shape)

    new_shape = (im.shape[0] // bin_size, im.shape[1])
    if new_shape[0] == 0 or new_shape[1] == 0:
        print("Cropped image cannot be binned because dimensions are too small")
        return   
    shape = (new_shape[0], im.shape[0] // new_shape[0],
             new_shape[1], im.shape[1] // new_shape[1])
    new_image = im.reshape(shape).mean(-1).mean(1)
    return new_image

In [13]:
def contrast_ratios(image):
    contrast_ratios = []
    image = np.rot90(image, k=3)
    for i in range(image.shape[0]):
        row = image[i]
        # calculate max and min pixel value in each row
        row_low = float(np.min(row))#np.quantile(row, 0.1)
        row_high = float(np.max(row))#np.quantile(row, 0.95)      
        # calculte contrast ratio of each row
        contrastratio = (row_high - row_low)/ (2* (row_high+row_low))
        #print(contrastratio)
        contrast_ratios.append(contrastratio)
    #normalize contrast ratios
    contrast_ratios = contrast_ratios / np.max(contrast_ratios)
    # determine what contrast counts as 'in focus' - this is 0.9 of the maximum
    cutoff = contrast_ratios > 0.9
    indices = np.nonzero(cutoff)
    lower = np.min(indices)
    higher = np.max(indices)
    in_focus_region = image[lower:higher,:]
    # returns the pixel length of the depth of field, the image cropped to only the in focus region of stripes, and an array of all the contrast ratios
    return higher - lower, in_focus_region, contrast_ratios

In [14]:
def calc_dof_len(in_dist, mag, pixel_size):
    return (in_dist * pixel_size) / mag

In [15]:
def configure_exposure(cam):
    """
     This function configures a custom exposure time. Automatic exposure is turned
     off in order to allow for the customization, and then the custom setting is
     applied.
    """

    print('*** CONFIGURING EXPOSURE ***\n')

    try:
        result = True

        if cam.ExposureAuto.GetAccessMode() != PySpin.RW:
            print('Unable to disable automatic exposure. Aborting...')
            return False

        cam.ExposureAuto.SetValue(PySpin.ExposureAuto_Off)
        print('Automatic exposure disabled...')

        if cam.ExposureTime.GetAccessMode() != PySpin.RW:
            print('Unable to set exposure time. Aborting...')
            return False

        # Ensure desired exposure time does not exceed the maximum
        exposure_time_to_set = EXPOSURE_TIME
        exposure_time_to_set = min(cam.ExposureTime.GetMax(), exposure_time_to_set)
        cam.ExposureTime.SetValue(exposure_time_to_set)
        print('Shutter time set to %s us...\n' % cam.ExposureTime.GetValue())

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        result = False

    return result

In [16]:
def reset_exposure(cam):
    """
    This function returns the camera to a normal state by re-enabling automatic exposure.
    """
    try:
        result = True

        if cam.ExposureAuto.GetAccessMode() != PySpin.RW:
            print('Unable to enable automatic exposure (node retrieval). Non-fatal error...')
            return False

        cam.ExposureAuto.SetValue(PySpin.ExposureAuto_Continuous)

        print('Automatic exposure enabled...')

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        result = False

    return result

In [17]:
def configure_gain(cam):

    print('*** CONFIGURING GAIN ***\n')

    try:
        result = True

        if cam.GainAuto.GetAccessMode() != PySpin.RW:
            print('Unable to disable automatic gain. Aborting...')
            return False

        cam.GainAuto.SetValue(PySpin.GainAuto_Off)
        print('Automatic gain disabled...')

    
        if cam.Gain.GetAccessMode() != PySpin.RW:
            print('Unable to set gain . Aborting...')
            return False

        # Ensure desired exposure time does not exceed the maximum
        gain_to_set = GAIN
        cam.Gain.SetValue(gain_to_set)
        print('Gain set to %s ...\n' % gain_to_set)

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        result = False

    return result

In [18]:
def reset_gain(cam):
    try:
        result = True

        if cam.GainAuto.GetAccessMode() != PySpin.RW:
            print('Unable to enable gain exposure (node retrieval). Non-fatal error...')
            return False

        cam.GainAuto.SetValue(PySpin.GainAuto_Continuous)

        print('Automatic gain enabled...')

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        result = False

    return result 

In [19]:
def setWidth(nodemap,width):
    result = 0
    node_width = PySpin.CIntegerPtr(nodemap.GetNode('Width'))
    if PySpin.IsAvailable(node_width) and PySpin.IsWritable(node_width):
        incrementRemainder = (width - node_width.GetMin()) % node_width.GetInc()
        if width > node_width.GetMax():
            width = node_width.GetMax()
        elif width < node_width.GetMin():
            width = node_width.GetMin()
        elif incrementRemainder !=0:
            width -= incrementRemainder
        node_width.SetValue(width)
        print('Width set to %i...' % node_width.GetValue())
        return 0

    else:
        print('Width not available...')
        return -1 

In [20]:
def setHeight(nodemap,height):
    result = 0
    node_height = PySpin.CIntegerPtr(nodemap.GetNode('Height'))
    if PySpin.IsAvailable(node_height) and PySpin.IsWritable(node_height):
        incrementRemainder = (height - node_height.GetMin())% node_height.GetInc()
        if height > node_height.GetMax():
            height = node_height.GetMax()
        elif height < node_height.GetMin():
            height = node_height.GetMin()
        elif incrementRemainder != 0:
            height -= incrementRemainder
        node_height.SetValue(height)
        print('Height set to %i...' % node_height.GetValue())
        return 0

    else:
        print('Height not available...')
        return -1


In [21]:
def factory_reset(cam):
    print('***FACTORY RESET***')
    nodemap = cam.GetNodeMap()
    node_factory_reset = PySpin.CCommandPtr(nodemap.GetNode('FactoryReset'))
    if not PySpin.IsAvailable(node_factory_reset) or not PySpin.IsWritable(node_factory_reset):
        print("Unable to access factory reset (enum retrieval). Aborting...")
    node_factory_reset.Execute()
    return True

In [22]:
%matplotlib inline

In [24]:
BIN_SIZE = 8 # bin size for binning image in mode 0
PIXEL_SIZE = .0024 # in mm
EXPOSURE_TIME = 400000 # in microseconds
GAIN = 0.0
MAGNIFICATION = 0.4
WIDTH = 1200
HEIGHT = 2000
NUM_ITERATIONS = 1
PLOT_SHOW_TIME = 15

%matplotlib qt
plt.ion()

def handle_close(evt):
    """
    This function will close the GUI when close event happens.

    :param evt: Event that occurs when the figure closes.
    :type evt: Event
    """

    global continue_recording
    continue_recording = False


def acquire_and_display_images(cam, nodemap, nodemap_tldevice):
    """
    This function continuously acquires images from a device and display them in a GUI.

    :param cam: Camera to acquire images from.
    :param nodemap: Device nodemap.
    :param nodemap_tldevice: Transport layer device nodemap.
    :type cam: CameraPtr
    :type nodemap: INodeMap
    :type nodemap_tldevice: INodeMap
    :return: True if successful, False otherwise.
    :rtype: bool
    """
    global continue_recording

    sNodemap = cam.GetTLStreamNodeMap()

    # Change bufferhandling mode to NewestOnly
    node_bufferhandling_mode = PySpin.CEnumerationPtr(sNodemap.GetNode('StreamBufferHandlingMode'))
    if not PySpin.IsAvailable(node_bufferhandling_mode) or not PySpin.IsWritable(node_bufferhandling_mode):
        print('Unable to set stream buffer handling mode.. Aborting...')
        return False

    # Retrieve entry node from enumeration node
    node_newestonly = node_bufferhandling_mode.GetEntryByName('NewestOnly')
    if not PySpin.IsAvailable(node_newestonly) or not PySpin.IsReadable(node_newestonly):
        print('Unable to set stream buffer handling mode.. Aborting...')
        return False

    # Retrieve integer value from entry node
    node_newestonly_mode = node_newestonly.GetValue()

    # Set integer value from entry node as new value of enumeration node
    node_bufferhandling_mode.SetIntValue(node_newestonly_mode)
    
    print('set stream buffer handling mode to newest only.')

    print('*** IMAGE ACQUISITION ***\n')
    try:
        node_acquisition_mode = PySpin.CEnumerationPtr(nodemap.GetNode('AcquisitionMode'))
        if not PySpin.IsAvailable(node_acquisition_mode) or not PySpin.IsWritable(node_acquisition_mode):
            print('Unable to set acquisition mode to continuous (enum retrieval). Aborting...')
            return False

        # Retrieve entry node from enumeration node
        node_acquisition_mode_continuous = node_acquisition_mode.GetEntryByName('Continuous')
        if not PySpin.IsAvailable(node_acquisition_mode_continuous) or not PySpin.IsReadable(
                node_acquisition_mode_continuous):
            print('Unable to set acquisition mode to continuous (entry retrieval). Aborting...')
            return False
        
        # Retrieve integer value from entry node
        acquisition_mode_continuous = node_acquisition_mode_continuous.GetValue()

        # Set integer value from entry node as new value of enumeration node
        node_acquisition_mode.SetIntValue(acquisition_mode_continuous)

        print('Acquisition mode set to continuous...')
        setWidth(nodemap, WIDTH)
        setHeight(nodemap, HEIGHT)
        # begin acquisition
        cam.BeginAcquisition()

        print('Acquiring images...')

        # retrieve device serial number
        device_serial_number = ''
        node_device_serial_number = PySpin.CStringPtr(nodemap_tldevice.GetNode('DeviceSerialNumber'))
        if PySpin.IsAvailable(node_device_serial_number) and PySpin.IsReadable(node_device_serial_number):
            device_serial_number = node_device_serial_number.GetValue()
            print('Device serial number retrieved as %s...' % device_serial_number)

        # Close program
        print('Press (and hold) enter to close the program...')
        
        i = 0
        
        while(i < NUM_ITERATIONS):
            try:
                dof_array = []

                #  Retrieve next received image
                #
                #  *** NOTES ***
                #  Capturing an image houses images on the camera buffer. Trying
                #  to capture an image that does not exist will hang the camera.
                #
                #  *** LATER ***
                #  Once an image from the buffer is saved and/or no longer
                #  needed, the image must be released in order to keep the
                #  buffer from filling up.
                
                image_result = cam.GetNextImage(1000)

                #  Ensure image completion
                if image_result.IsIncomplete():
                    print('Image incomplete with image status %d ...' % image_result.GetImageStatus())

                else:
                    
                    # Getting the image data as a numpy array
                    image_data = image_result.GetNDArray() 
                    image_data = np.rot90(image_data)
                    plt.imshow(image_data, cmap= "gray")
                    plt.title("Original Image")
                    Image.fromarray(image_data.astype(np.uint8)).save("original image dof.png")
                    # create a FIGURE inside which all data required and image can be viewed side by side 
                    fig = plt.figure(layout='constrained')
                    
                    # separate the figure into two subplots 
                    ax_arrays = fig.subplots(1, 3, squeeze=False)
                    ax1 = ax_arrays[0,1]
                    ax2 = ax_arrays[0,2]
                    ax3 = ax_arrays[0,0]
                    cropped = get_cropped_region(image_data)
                       
                    if cropped is not None and cropped.size!=0:
                        ax3.imshow(cropped, cmap = "gray")
                        ax3.set_title("Cropped image")
                        cropped = bin_image(cropped, BIN_SIZE)
                        if cropped is not None and cropped.size != 0:
                            dof_in_pixels = contrast_ratios(cropped)
                            # plot the in focus region of the current image
                            ax1.imshow(dof_in_pixels[1], cmap='gray')
                            ax1.set_title("In focus region")
                            # plot contrast ratios for one image
                            ax2.plot(dof_in_pixels[2])
                            ax2.set_ylabel('Contrast ratios')
                            print("DOF in pixels:")
                            print(dof_in_pixels[0])
                            print("DOF")
                            dof_actual = calc_dof_len(dof_in_pixels[0], PIXEL_SIZE, MAGNIFICATION)
                            print(dof_actual)
                            plt.savefig("DOF calculation.png", format = 'png')
                            plt.pause(PLOT_SHOW_TIME)
                            plt.close()
                        else:
                            print("cropped image has size 0")
                    i += 1

                    # If user presses enter, close the program
                    if keyboard.is_pressed('ENTER'):
                        print('Program is closing...')
                        
                        # Close figure
                        plt.close('all')             
                        input('Done! Press Enter to exit...')
                        continue_recording=False                        
            
                #  Release image
                #
                #  *** NOTES ***
                #  Images retrieved directly from the camera (i.e. non-converted
                #  images) need to be released in order to keep from filling the
                #  buffer.
                image_result.Release()

            except PySpin.SpinnakerException as ex:
                print('Error: %s' % ex)
                return False
        #plt.plot(dof_array) 
        #plt.title("Dofs for each image")
        #  End acquisition
        #
        #  *** NOTES ***
        #  Ending acquisition appropriately helps ensure that devices clean up
        #  properly and do not need to be power-cycled to maintain integrity.
        cam.EndAcquisition()

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        return False
    return True


def run_single_camera(cam):
    """
    This function acts as the body of the example; please see NodeMapInfo example
    for more in-depth comments on setting up cameras.

    :param cam: Camera to run on.
    :type cam: CameraPtr
    :return: True if successful, False otherwise.
    :rtype: bool
    """
    try:
        result = True

        nodemap_tldevice = cam.GetTLDeviceNodeMap()

        # Initialize camera
        cam.Init()
        
        nodemap = cam.GetNodeMap()

        if not configure_gain(cam):
            return False
        if not configure_exposure(cam):
            return False
        
        # cam.SensorShutterMode.SetValue(PySpin.SensorShutterMode_Rolling)
        # cam.GammaEnable.SetValue(False)
        # cam.PixelFormat.SetValue(PySpin.PixelFormat_Mono16)
        
        # Retrieve GenICam nodemap        
        # Acquire images
        #uncomment factory_reset and reset the camera if the camera has beeen set to wait for a hardware trigger(i.e., if prepare camera has been run)
        # otherwise, there will be a NEW_EVENT_BUFFER error
        #factory_reset(cam)
        result &= acquire_and_display_images(cam, nodemap, nodemap_tldevice)
        
        # Reset exposure
        result &= reset_exposure(cam)
        result &= reset_gain(cam)
        print("About to deinitialize")
        # Deinitialize camera
        cam.DeInit()
        print("Deinitialized")

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        result = False

    return result


def main():
    """
    Example entry point; notice the volume of data that the logging event handler
    prints out on debug despite the fact that very little really happens in this
    example. Because of this, it may be better to have the logger set to lower
    level in order to provide a more concise, focused log.

    :return: True if successful, False otherwise.
    :rtype: bool
    """
    result = True
    # Retrieve singleton reference to system object
    system = PySpin.System.GetInstance()

    # Get current library version
    version = system.GetLibraryVersion()
    print('Library version: %d.%d.%d.%d' % (version.major, version.minor, version.type, version.build))

    # Retrieve list of cameras from the system
    cam_list = system.GetCameras()

    num_cameras = cam_list.GetSize()

    print('Number of cameras detected: %d' % num_cameras)

    # Finish if there are no cameras
    if num_cameras == 0:

        # Clear camera list before releasing system
        cam_list.Clear()

        # Release system instance
        system.ReleaseInstance()

        print('Not enough cameras!')
        input('Done! Press Enter to exit...')
        return False

    # Run example on each camera
    for i, cam in enumerate(cam_list):
        print('Running example for camera %d...' % i)

        result &= run_single_camera(cam)
        print('Camera %d example complete... \n' % i)

    # Release reference to camera
    # NOTE: Unlike the C++ ex amples, we cannot rely on pointer objects being automatically
    # cleaned up when going out of scope.
    # The usage of del is preferred to assigning the variable to None.
    del cam
    

    # Clear camera list before releasing system
    cam_list.Clear()
    

    # Release system }instance
    system.ReleaseInstance()
    return result

if __name__ == '__main__':
    if main():
        sys.exit(0)

    else:
        sys.exit(1)

Library version: 2.7.0.128
Number of cameras detected: 1
Running example for camera 0...
*** CONFIGURING GAIN ***

Automatic gain disabled...
Gain set to 0.0 ...

*** CONFIGURING EXPOSURE ***

Automatic exposure disabled...
Shutter time set to 400000.0 us...

set stream buffer handling mode to newest only.
*** IMAGE ACQUISITION ***

Acquisition mode set to continuous...
Width set to 1200...
Height set to 2000...
Acquiring images...
Device serial number retrieved as 22158653...
Press (and hold) enter to close the program...
(582, 2000)
(576, 2000)
DOF in pixels:
1253
DOF
208833.33333333337


Exception in thread Thread-5:
Traceback (most recent call last):
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/karolyn/Library/Python/3.8/lib/python/site-packages/keyboard/__init__.py", line 294, in listen
    _os_keyboard.listen(self.direct_callback)
  File "/Users/karolyn/Library/Python/3.8/lib/python/site-packages/keyboard/_darwinkeyboard.py", line 430, in listen
    raise OSError("Error 13 - Must be run as administrator")
OSError: Error 13 - Must be run as administrator


Automatic exposure enabled...
Automatic gain enabled...
About to deinitialize
Deinitialized
Camera 0 example complete... 



SystemExit: 0

/Users/karolyn/Library/Python/3.8/lib/python/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
#notes:
# If image is too dark, it will have trouble finding lines - increase exposure
# can go to about 500,000 us before starting to get errors
# If it crops out too much, the num_peaks paramaeter in the find_hough_lines cell that is passed into hough_line can be adjusted. Increasing num_peaks should 
# give a wider crop, and decreasing it should make it narrower
# If images don't save, close and reopen the notebook
# If there is not enough avilable memory to allocate buffers for streaming(Spinnaker error 1016), clear out RAM by quitting other processes, and restart the kernel
# restarting the kernel every time the error occur should work
# if the error still occurs, decrease image size by adjusting HEIGHT and WIDTH
# in order to get a good crop, image WIDTH should be decreased, since the 25mm camera will take a picture that includes multiple sets of stripes
# If program runs into runtime error, camera has to be power cycled to stop acquisition. Otherwise, there will be a Spinnaker error next time you run the program: Can't
# de-initialize camera, Image cleanup required as images are locked in the queue [-1004]


In [ ]:
%tb